Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [19]:
import torch
import rlcard
import json
import numpy as np
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent, CFRAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
set_global_seed(0)

In [5]:
eval_env = rlcard.make('limit-holdem', config={'seed': 0})

In [6]:
dqn_agent_2 = DQNAgent(
    eval_env.action_num,
    eval_env.state_shape[0],
    hidden_neurons=[1024, 512, 1024, 512]
)

dqn_agent_2.load(os.path.join('./models/limit_holdem_dqn/02/', 'step-2631447.pt'))

In [7]:
dqn_agent_3 = DQNAgent(
    eval_env.action_num,
    eval_env.state_shape[0],
    hidden_neurons=[1024, 512, 1024, 512]
)

dqn_agent_3.load(os.path.join('./models/limit_holdem_dqn/03/', 'step-5474450.pt'))

In [13]:
dqn_agent_4 = DQNAgent(
    eval_env.action_num,
    eval_env.state_shape[0],
    hidden_neurons=[1024, 512, 1024, 512]
)

dqn_agent_3.load(os.path.join('./models/limit_holdem_dqn/03/', 'step-2500210.pt'))

In [8]:
random_agent = RandomAgent(action_num=eval_env.action_num)

In [9]:
def eval(agent1, agent2):
    eval_env.set_agents([agent1, agent2])
    return tournament(eval_env, 100000)

In [10]:
eval(dqn_agent_2, dqn_agent_3)

[-0.28557, 0.28557]

In [11]:
eval(dqn_agent_3, dqn_agent_2)

[0.33526, -0.33526]

In [12]:
eval(dqn_agent_3, random_agent)

[1.7403, -1.7403]

In [14]:
eval(dqn_agent_2, dqn_agent_4)

[5.76854, -5.76854]

In [15]:
eval(dqn_agent_3, dqn_agent_4)

[4.09704, -4.09704]

In [45]:
dqn_agent_3.policy_net.eval()

with open('rlcard/rlcard/games/limitholdem/card2index.json', 'r') as file:
    card2index = json.load(file)

ai = { a: i for i, a in enumerate(eval_env.actions)}

In [46]:
obs = np.zeros(228)

obs[card2index['S2']] = 1
obs[card2index['C7']] = 1
obs[208] = 1
obs[213] = 1
obs[218] = 1
obs[223] = 1

state = {
    'obs': obs,
    'legal_actions': [ai['call'], ai['raise'], ai['fold']]
}

model_input = torch.tensor(state['obs'],
                           dtype=torch.float,
                           device=dqn_agent_3.device)

dqn_agent_3.policy_net(model_input)

tensor([ 0.0539,  0.4436, -5.2679,  0.1856], grad_fn=<AddBackward0>)